## Web Application using Spot Fleet

In this workshop we will extend the basics of AWS EC2 we learned from the `intro_to_aws` workshop by utilizing EC2 Spot Fleet instances for hosting the web application. Python is used extensively so you will need experience in or be comfortable reading python code. 


### Initialize notebook

We will be using the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) library for creation of all resources.

In [ ]:
import boto3
import sys
import os
import json
import base64
import project_path # path to helper methods

from lib import workshop
from botocore.exceptions import ClientError

ec2_client = boto3.client('ec2')
ec2 = boto3.resource('ec2')
alb_client = boto3.client("elbv2")
cloudwatch = boto3.client('cloudwatch')
ssm = boto3.client('ssm')
iam = boto3.client('iam')
cwe = boto3.client('events')
aa = boto3.client('application-autoscaling')
lambda_client = boto3.client('lambda')

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

workshop_user = 'spot' # no capitals all lower case
alb_name = 'alb-web-{0}'.format(workshop_user)
alb_sec_group_name = 'alb-sg-{0}'.format(workshop_user)
alb_target_group_name = 'alb-target-group-{0}'.format(workshop_user)
auto_scaling_group_name = 'web-asg-{0}'.format(workshop_user)
scale_up_name = 'scale_up_{0}'.format(workshop_user)
scale_down_name = 'scale_down_{0}'.format(workshop_user)
launch_template = 'webserver-lt-{0}'.format(workshop_user)
root_volume_size = 2000

use_existing = True

### [Create VPC](https://aws.amazon.com/vpc/)

Amazon Virtual Private Cloud (Amazon VPC) lets you provision a logically isolated section of the AWS Cloud where you can launch AWS resources in a virtual network that you define. You have complete control over your virtual networking environment, including selection of your own IP address range, creation of subnets, and configuration of route tables and network gateways. You can use both IPv4 and IPv6 in your VPC for secure and easy access to resources and applications.

In [ ]:
if use_existing:
    vpc_filter = [{'Name':'isDefault', 'Values':['true']}]
    default_vpc = ec2_client.describe_vpcs(Filters=vpc_filter)
    vpc_id = default_vpc['Vpcs'][0]['VpcId']

    subnet_filter = [{'Name':'vpc-id', 'Values':[vpc_id]}]
    subnets = ec2_client.describe_subnets(Filters=subnet_filter)
    subnet1_id = subnets['Subnets'][0]['SubnetId']
    subnet2_id = subnets['Subnets'][1]['SubnetId']
else: 
    vpc, subnet1, subnet2 = workshop.create_and_configure_vpc()
    vpc_id = vpc.id
    subnet1_id = subnet1.id
    subnet2_id = subnet2.id

In [ ]:
print(vpc_id)
print(subnet1_id)
print(subnet2_id)
print(region)

### [Create Security Groups](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_SecurityGroups.html)


A security group acts as a virtual firewall for your instance to control inbound and outbound traffic. When you launch an instance in a VPC, you can assign up to five security groups to the instance. Security groups act at the instance level, not the subnet level. Therefore, each instance in a subnet in your VPC could be assigned to a different set of security groups. If you don't specify a particular group at launch time, the instance is automatically assigned to the default security group for the VPC.

[ec2_client.create_security_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.create_security_group) boto3 documentation

In [ ]:
sg = ec2_client.create_security_group(
    Description='security group for ALB',
    GroupName=alb_sec_group_name,
    VpcId=vpc_id
)
alb_sec_group_id=sg["GroupId"]
print('ALB security group id - ' + alb_sec_group_id)

### Configure available ports

In order for the ALB to communicate with the outside world, we will open port 80. As you can see in the call below we can define the `ToPort` and `FromPort` and a `CidrIp` range we want to allow.

[ec2_client.authorize_security_group_ingress](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.authorize_security_group_ingress) boto3 documentation

In [ ]:
data = ec2_client.authorize_security_group_ingress(
    GroupId=alb_sec_group_id,
    IpPermissions=[
        {'IpProtocol': 'tcp',
         'FromPort': 80,
         'ToPort': 80,
         'IpRanges': [
            {
                'CidrIp': '0.0.0.0/0',
                'Description': 'HTTP access'
            },
          ]
        }
    ]
)

### [Create Application Load Balancer (ALB)](https://docs.aws.amazon.com/elasticloadbalancing/latest/application/introduction.html)


Elastic Load Balancing supports three types of load balancers: Application Load Balancers, Network Load Balancers, and Classic Load Balancers. In this example we will be using an [Application Load Balancers](https://docs.aws.amazon.com/elasticloadbalancing/latest/application/introduction.html). For more information about Network Load Balancers, see the [User Guide for Network Load Balancers](https://docs.aws.amazon.com/elasticloadbalancing/latest/network/). For more information about Classic Load Balancers, see the [User Guide for Classic Load Balancers](https://docs.aws.amazon.com/elasticloadbalancing/latest/classic/).

[elbv2.create_load_balancer](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/elbv2.html#ElasticLoadBalancingv2.Client.create_load_balancer) boto3 documentation

In [ ]:
alb = alb_client.create_load_balancer(
    Name=alb_name,
    Subnets=[
        subnet1_id,
        subnet2_id
    ],
    SecurityGroups=[
        alb_sec_group_id,
    ],
    Scheme='internet-facing',
    Type='application',
    IpAddressType='ipv4'
)

alb_arn = alb["LoadBalancers"][0]["LoadBalancerArn"]
alb_name = alb["LoadBalancers"][0]["LoadBalancerName"]
print(alb_arn)
print(alb_name)

### [Create Target Group](https://docs.aws.amazon.com/elasticloadbalancing/latest/application/load-balancer-target-groups.html)

Each target group is used to route requests to one or more registered targets. When you create each listener rule, you specify a target group and conditions. When a rule condition is met, traffic is forwarded to the corresponding target group. You can create different target groups for different types of requests. For example, create one target group for general requests and other target groups for requests to the microservices for your application.

[elbv2.create_target_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/elbv2.html#ElasticLoadBalancingv2.Client.create_target_group) boto3 documentation

In [ ]:
target_group = alb_client.create_target_group(
    Name=alb_target_group_name,
    Protocol='HTTP',
    Port=80,
    VpcId=vpc_id,
    HealthCheckProtocol='HTTP',
    HealthCheckPort='80',
    HealthCheckPath='/'
)

target_group_arn = target_group["TargetGroups"][0]["TargetGroupArn"]
print(target_group_arn)

### [Create Listener](https://docs.aws.amazon.com/elasticloadbalancing/latest/application/load-balancer-listeners.html)

Before you start using your Application Load Balancer, you must add one or more listeners. A listener is a process that checks for connection requests, using the protocol and port that you configure. The rules that you define for a listener determine how the load balancer routes requests to the targets in one or more target groups.

[elbv2.create_listener](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/elbv2.html#ElasticLoadBalancingv2.Client.create_listener) boto3 documentation

In [ ]:
listener = alb_client.create_listener(
    DefaultActions=[
        {'TargetGroupArn': target_group_arn,
         'Type': 'forward'
        }],
    LoadBalancerArn=alb_arn,
    Port=80,
    Protocol='HTTP'
)

listener_arn = listener["Listeners"][0]["ListenerArn"]
print(listener_arn)

### Get Latest [Amazon Linux AMI](https://aws.amazon.com/amazon-linux-ami/)

The Amazon Linux 2 AMI is a supported and maintained Linux image provided by Amazon Web Services for use on Amazon Elastic Compute Cloud (Amazon EC2). It is designed to provide a stable, secure, and high performance execution environment for applications running on Amazon EC2. It supports the latest EC2 instance type features and includes packages that enable easy integration with AWS. Amazon Web Services provides ongoing security and maintenance updates to all instances running the Amazon Linux AMI. The Amazon Linux AMI is provided at no additional charge to Amazon EC2 users. 

In [ ]:
response = ssm.get_parameters(Names=['/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'])
ami = response['Parameters'][0]['Value']
print(ami)

### Create Spot Fleet IAM Role

In [ ]:
role_doc = {
        "Version": "2012-10-17", 
        "Statement": [
            {"Sid": "", 
             "Effect": "Allow", 
             "Principal": {
                 "Service": "spotfleet.amazonaws.com"
             }, 
             "Action": "sts:AssumeRole"
        }]
    }

role_name = 'WorkshopSpotFleetRole-{0}'.format(workshop_user)
spot_fleet_role_arn = workshop.create_role(iam=iam, policy_name=role_name, \
                                           assume_role_policy_document=json.dumps(role_doc), \
                                           managed_policy='arn:aws:iam::aws:policy/service-role/AmazonEC2SpotFleetRole')

# Include SSM role to be able to shell into the instances
iam.attach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEC2RoleforSSM')
print(spot_fleet_role_arn)

### Create UserData to install Apache web server and download index


In [ ]:
%%writefile userdata.sh
#!/bin/bash
yum -y update
yum -y install httpd
chkconfig httpd on
instanceid=$(curl http://169.254.169.254/latest/meta-data/instance-id)
echo "hello spot workshop from $instanceid" > /var/www/html/index.html
service httpd start

### Load userdata.sh

We will read the UserData into a local variable and base64 encode the contents of the file to be used on the EC2 instance launch configuraton.

In [ ]:
fh=open("userdata.sh")
userdata=fh.read()
fh.close()

userdataencode = base64.b64encode(userdata.encode()).decode("ascii")

### [Create Instance Profile for EC2 instances](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-ec2_instance-profiles.html)

An instance profile is a container for an IAM role that you can use to pass role information to an EC2 instance when the instance starts.

[iam.create_instance_profile](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.create_instance_profile)

In [ ]:
instance_profile_name = 'SpotSessionManagerAccessRole'

response = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)

instance_profile_arn = response['InstanceProfile']['Arn']
print(instance_profile_arn)

Add the managed Role `SessionManagerAccessRole` to allow Session Manager to gain shell access to the EC2 instances for troubleshooting.

In [ ]:
iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName='SessionManagerAccessRole'
)

### [Create Launch Template](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-launch-templates.html)

You can create a launch template that contains the configuration information to launch an instance. Launch templates enable you to store launch parameters so that you do not have to specify them every time you launch an instance. 

[ec2_client.create_launch_template](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.create_launch_template)

In [ ]:
response = ec2_client.create_launch_template(
    LaunchTemplateName=launch_template,
    LaunchTemplateData={
        'IamInstanceProfile': {
            'Arn': instance_profile_arn,
        },
        'BlockDeviceMappings': [
            {
                'DeviceName': '/dev/xvda',
                'Ebs': {
                    'DeleteOnTermination': True,
                    'VolumeSize': root_volume_size,
                    'VolumeType': 'gp2'
                }
            }
        ],
        'UserData': userdataencode,
        'ImageId': ami,
        'SecurityGroupIds': [
            alb_sec_group_id,
        ],
    }
)

launch_template_id = response['LaunchTemplate']['LaunchTemplateId']
print(launch_template_id)

### [Create EC2 Spot Fleet](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/spot-fleet.html)

A Spot Fleet is a collection, or fleet, of Spot Instances, and optionally On-Demand Instances. The request for Spot Instances is fulfilled if the maximum price you specified in the request exceeds the current Spot price and there is available capacity.
 
You can set the `SpotPrice` on the `SpotFleetRequestConfig` to lower how much you are willing to pay, but by default it will use the `On-Demand` price of the instance type selected and you will only pay for the current going market rate of the instance.

[ec2_client.request_spot_fleet](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.request_spot_fleet)

In [ ]:
response = ec2_client.request_spot_fleet(
    SpotFleetRequestConfig={
        'AllocationStrategy': 'lowestPrice',
        'OnDemandAllocationStrategy': 'lowestPrice',
        'IamFleetRole': spot_fleet_role_arn,     
        'LaunchTemplateConfigs': [
            {
                'LaunchTemplateSpecification': {
                    'LaunchTemplateId': launch_template_id,
                    'Version': '1'
                },
                'Overrides': [
                    {
                        'InstanceType': 't3.micro'
                    },
                ]
            },
        ],
        'TargetCapacity': 3,
        'TerminateInstancesWithExpiration': True,
        'Type': 'maintain',
        'ReplaceUnhealthyInstances': True,
        'InstanceInterruptionBehavior': 'terminate',
        'LoadBalancersConfig': {
            'TargetGroupsConfig': {
                'TargetGroups': [
                    {
                        'Arn': target_group_arn
                    }
                ]
            }
        }
    }
)

spot_request_id = response['SpotFleetRequestId']
print(spot_request_id)

### Wait for Web servers to start

Once the spot request has been fulfilled you will be able to select the spot fleet request and view the savings tab like below:

![Spot Savings](../../docs/assets/images/spot-savings.png)

### [Register target for application autoscaling](https://docs.aws.amazon.com/autoscaling/application/APIReference/API_RegisterScalableTarget.html)

Registers or updates a scalable target. A scalable target is a resource that Application Auto Scaling can scale out and scale in. Each scalable target has a resource ID, scalable dimension, and namespace, as well as values for minimum and maximum capacity.

[aa.register_scalable_target](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/application-autoscaling.html#ApplicationAutoScaling.Client.register_scalable_target)

In [ ]:
resource_id = 'spot-fleet-request/{0}'.format(spot_request_id)

response = aa.register_scalable_target(
    ServiceNamespace='ec2',
    ResourceId=resource_id,
    ScalableDimension='ec2:spot-fleet-request:TargetCapacity',
    MinCapacity=3,
    MaxCapacity=10
)

### [Application Autoscaling with Target Tracking](https://docs.aws.amazon.com/autoscaling/application/userguide/application-auto-scaling-target-tracking.html)

With target tracking scaling policies, you choose a scaling metric and set a target value. Application Auto Scaling creates and manages the CloudWatch alarms that trigger the scaling policy and calculates the scaling adjustment based on the metric and the target value. The scaling policy adds or removes capacity as required to keep the metric at, or close to, the specified target value. In addition to keeping the metric close to the target value, a target tracking scaling policy also adjusts to changes in the metric due to a changing load pattern.

[aa.put_scaling_policy](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/application-autoscaling.html#ApplicationAutoScaling.Client.put_scaling_policy)

In [ ]:
response = aa.put_scaling_policy(
    PolicyName='Scale fleet to target',
    ServiceNamespace='ec2',
    ResourceId= resource_id,
    ScalableDimension='ec2:spot-fleet-request:TargetCapacity',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 50,
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'EC2SpotFleetRequestAverageCPUUtilization'
        },
        'ScaleOutCooldown': 300,
        'DisableScaleIn': False
    }
)

print(response['PolicyARN'])

### [Spot Pricing History](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-spot-instances-history.html)

Let's take a look at the spot price history available to you to beter understand the cost savings and availability of the instances. Once the page is loaded click on the `Pricing History` button to review the spot pricing history for the `t3.micro` instances.

In [ ]:
print('https://{0}.console.aws.amazon.com/ec2sp/v1/spot/home?region={0}#'.format(region))

### View Target Group

The `Target Group` for the spot fleet request will be used with the ALB. We will be waiting for the instances to get into a `healthy` state and readily available to serve up the index page.

In [ ]:
print('https://{0}.console.aws.amazon.com/ec2/v2/home?region={0}#TargetGroups:sort=targetGroupName'.format(region))

### View Web Application

If all has gone well to this point you should be able to view the ALB and see the request being routed to different spot instances behind the load balancer.

In [ ]:
print('http://{0}'.format(alb['LoadBalancers'][0]['DNSName']))

### [Spot Instance Interruption Handler](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/spot-interruptions.html)

Demand for Spot Instances can vary significantly from moment to moment, and the availability of Spot Instances can also vary significantly depending on how many unused EC2 instances are available. It is always possible that your Spot Instance might be interrupted. Therefore, you must ensure that your application is prepared for a Spot Instance interruption.

We will create a Lambda function that will be triggered from an CloudWatch Event rule on EC2 state-change notifications. The `event` form CloudWatch will contain the `instance-id` and the `instance-action` and you can decide what intervention is needed if any. The example code below logs typically calls to get the details of the EC2 instance, an example of deregistering with the alb if necessary, and logs the details trhoughout. 

In [ ]:
%%writefile spothandler.py
import boto3

def handler(event, context):
    print(event['detail'])
    instanceId = event['detail']['instance-id']
    instanceAction = event['detail']['instance-action']
    
    print("Interrupting instance:")    
    print("{0}, {1}".format(instanceId, instanceAction))
    return

### Zip Lambda function for Spot notifications

In [ ]:
%%bash
zip spot-notify.zip -r6 spothandler.py

### Create the Lamba function to trigger the orchestration

[lambda_client.create_function](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda.html#Lambda.Client.create_function)

In [ ]:
spot_interrupt_fn_name = "Spot-Notify-{0}".format(workshop_user)
lambda_role_arn = 'arn:aws:iam::{0}:role/lambda_basic_execution'.format(account_id)

response = lambda_client.create_function(
    FunctionName=spot_interrupt_fn_name,
    Runtime='python2.7',
    Role=lambda_role_arn,
    Handler="spothandler.handler",
    Code={'ZipFile': open("spot-notify.zip", 'rb').read(), },
)

In [ ]:
lambda_arn = response['FunctionArn']
print(lambda_arn)

### [Create CloudWatch Rule](https://docs.aws.amazon.com/AmazonCloudWatch/latest/events/WhatIsCloudWatchEvents.html)

Amazon CloudWatch Events delivers a near real-time stream of system events that describe changes in Amazon Web Services (AWS) resources. In this instance we will trigger an event based on EC2 state changes.

[cwe.put_rule](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html#CloudWatchEvents.Client.put_rule)

In [ ]:
spot_rule = 'Spot-Notification-{0}'.format(workshop_user)
response= cwe.put_rule(
    Name=spot_rule,
    EventPattern=json.dumps(
        {
            "source": ["aws.ec2"],
            "detail-type": ["EC2 Spot Instance Interruption Warning"]
        }),
    State='ENABLED',
    Description='EC2 Spot Event Change Rule'
)
    
rule_arn = response['RuleArn']
print(rule_arn)

### [Add Permission to Lambda function](https://docs.aws.amazon.com/lambda/latest/dg/API_AddPermission.html)

Grants an AWS service or another account permission to use a function. You can apply the policy at the function level, or specify a qualifier to restrict access to a single version or alias. In this example, we will be granting permission to CloudWatch events to trigger the lambda function.

In [ ]:
lambda_client.add_permission(
    FunctionName=spot_interrupt_fn_name,
    StatementId="{0}-Event".format(fn_name),
    Action='lambda:InvokeFunction',
    Principal='events.amazonaws.com',
    SourceArn=rule_arn,
)

### Create Target for Rule

Targets need to be configured when a CloudWatch rule is raised. Here we will add the Lambda function that handles the Spot interruptions as the target for the rule to allow you to act on the instance before termination.

[cwe.put_targets](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html#CloudWatchEvents.Client.put_targets)

In [ ]:
response = cwe.put_targets(
    Rule=spot_rule,
    Targets=[
        {
            'Id': spot_rule,
            'Arn': spot_arn,
        }
    ]
)

### View Lambda function for Spot interruptions

![Lambda Events](../../docs/assets/images/lambda-designer.png)

In [ ]:
print('https://{0}.console.aws.amazon.com/lambda/home?region={0}#/functions/{1}?tab=graph'.format(region, fn_name))

### Remove instance from Spot Fleet request

In the link below, find the splot fleet request created above and check the checkbox for the row.

![Spot Req](../../docs/assets/images/spot-fleet-req.png)

---

Find the `Auto Scaling` tab and click the `Edit` button an change the `Scale capacity between` min from `3` to `2` and click `Save`. Once the spot fleet request is updated it will trigger the CloudWatch rule and trigger the Lambda function.

---

![Spot AS](../../docs/assets/images/spot-fleet-as.png)

In [ ]:
print('https://{0}.console.aws.amazon.com/ec2sp/v1/spot/home?region={0}#'.format(region))

### View CloudWatch logs on Spot Interruption Lambda Function

On the Lambda function select the `Monitoring` tab and click the `View logs in CloudWatch` button to see the execution of the Lambda function. When the notification is sent to CloudWatch it will look similar to below:

```json
{
    "version": "0",
    "id": "12345678-1234-1234-1234-123456789012",
    "detail-type": "EC2 Spot Instance Interruption Warning",
    "source": "aws.ec2",
    "account": "123456789012",
    "time": "yyyy-mm-ddThh:mm:ssZ",
    "region": "us-east-2",
    "resources": ["arn:aws:ec2:us-east-2:123456789012:instance/i-1234567890abcdef0"],
    "detail": {
        "instance-id": "i-1234567890abcdef0",
        "instance-action": "action"
    }
}
```

In [ ]:
print('https://{0}.console.aws.amazon.com/lambda/home?region={0}#/functions/{1}?tab=graph'.format(region, fn_name))

You have now walked through creating a web application backed by spot instances and what it takes to monitor for spot interruptions. If you are interested in learning more about EC2 spot follow this [link](https://aws.amazon.com/ec2/spot/).

## Clean Up

In order to remove everything created in this workshop you can run the cells below and finally remove the VPC created for this workshop.

In [ ]:
response = ec2_client.cancel_spot_fleet_requests(
    SpotFleetRequestIds=[
        spot_request_id,
    ],
    TerminateInstances=True
)

In [ ]:
response = ec2_client.delete_launch_template(LaunchTemplateId=launch_template_id)

In [ ]:
response = iam.remove_role_from_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName='SessionManagerAccessRole'
)

In [ ]:
response = iam.delete_instance_profile(
    InstanceProfileName=instance_profile_name
)

In [ ]:
response = alb_client.delete_listener(ListenerArn=listener_arn)

In [ ]:
response = alb_client.delete_target_group(TargetGroupArn=target_group_arn)

In [ ]:
response = alb_client.delete_load_balancer(LoadBalancerArn=alb_arn)

In [ ]:
response = iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEC2SpotFleetRole'
)

response = iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEC2RoleforSSM'
)

In [ ]:
response = iam.delete_role(RoleName=role_name)

In [ ]:
response = lambda_client.delete_function(FunctionName=spot_interrupt_fn_name)

In [ ]:
response = ec2_client.delete_security_group(GroupId=alb_sec_group_id)

In [ ]:
print(spot_interrupt_fn_name)

In [ ]:
response = cwe.remove_targets(
    Rule=spot_rule,
    Ids=[
        spot_rule,
    ],
    Force=True
)

In [ ]:
response = cwe.delete_rule(
    Name=spot_rule,
    Force=True
)

In [ ]:
response = lambda_client.delete_function(FunctionName=fn_name)

In [ ]:
response = aa.delete_scaling_policy(
    PolicyName='Scale fleet to target',
    ServiceNamespace='ec2',
    ResourceId=resource_id,
    ScalableDimension='ec2:spot-fleet-request:TargetCapacity'
)